In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

tf.__version__

/Users/seb/.local/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/seb/.local/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/seb/.local/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/seb/.local/lib/python3.7/site-packages/tensorboard/compat/tensorflow_

'2.0.0'

## Load Data

In [2]:
train_set=pd.read_csv('../../data/generated/interactions_train_alt.csv').dropna()
test_matrix=pd.read_csv('../../data/generated/interactions_test_alt.csv').dropna()
recipes = pd.read_csv('../../data/generated/recipes.csv')

full_data = pd.concat([train_set, test_matrix])

full_matrix = full_data.pivot_table(index='u', columns='i', values='rating', dropna=False)
print(f'Shape of fullMatrix User-Movie-Matrix:\t{full_matrix.shape}')

Shape of fullMatrix User-Movie-Matrix:	(6384, 3148)


In [3]:
#remplace rating from 0 to 5 to a boolean information : > 3 like (1), otherwise dislike (0)
full_matrix = full_matrix.applymap(lambda x : x if np.isnan(x) else int(x>3))

In [4]:
#replace all missing rating by -1 (as rating are from 0 to 5)
#the -1 will be then used in the model loss function as a mask
full_matrix.fillna(-1, inplace=True)

## Load the model trained on Colab

In [5]:
my_model = tf.keras.models.load_model('ae_v3.h5')

W1127 22:50:36.899568 4652324288 hdf5_format.py:177] No training configuration found in save file: the model was *not* compiled. Compile it manually.


## get recommandations from one user

In [13]:
#getting user ratings
#id = 24240
id=33

user = full_matrix[full_matrix.index == id].T
user_ratings = user[user[id]!=-1]


In [14]:
#get predictions from model
user = full_matrix[full_matrix.index == id].values
preds = my_model.predict(user, verbose=1)
preds = pd.DataFrame(preds, columns = full_matrix.columns)

1/1 [==============================] - 0s 2ms/sample


In [15]:
print('recipes rated by user in set')

summary = user_ratings.join(recipes[recipes['i']
                                .isin(user_ratings.T.columns)][['i','name']].set_index('i'))

summary = summary.join(preds.T)
summary.columns = ['rating', 'name', 'pred']

#get user mean square error
print('MSE for user', id, ':', np.square(preds[user_ratings.T.columns].values - user_ratings.T.values)
      .mean())

display(summary[['name', 'rating', 'pred']].sort_values('pred', ascending=False))

recipes rated by user in set
MSE for user 33 : 0.017404567298738714


,name,rating,pred
i,,,
97450,roasted parmesan garlic cauliflower,1.0,1.052626
149428,wonderful salsa,1.0,1.017623
129723,spinach garlic pizza,1.0,1.016191
2949,oreo balls,1.0,1.010944
78451,crispy coconut chicken fingers,1.0,0.989550
87565,chicken pesto,1.0,0.984327
135400,payday bars,1.0,0.974475
101200,tortellini tomato spinach soup,1.0,0.955344
19812,simply sour cream chicken enchiladas,1.0,0.943128


In [16]:
#top 10 recipes
reco = preds[set(preds.columns) - set (user_ratings.columns)].T.sort_values([0],ascending=False)
reco_top10 = reco.head(10)
reco_top10 = reco_top10.join(recipes[['i','name']].set_index('i'))
reco_top10.columns = ['pred', 'name']
print('The top 10 recommandations from Auto Encoder are :')
display(reco_top10[['name', 'pred']])

The top 10 recommandations from Auto Encoder are :


,name,pred
i,,
37397,zesty salsa for canning,1.654163
31342,quinoa stir fry with spinach walnuts,1.584497
148545,dessert cheese ball,1.530520
106867,steak wine marinade,1.520369
96750,butternut squash fries,1.509151
44513,minado s perfect sushi rice,1.481694
126036,no pudge brownie clone,1.462906
14650,mititei small ground beef sausages,1.452775
101550,thanksgiving stuffing cheat using stove top,1.440389


In [17]:
#flop 10 recipes
reco = preds[set(preds.columns) - set (user_ratings.columns)].T.sort_values([0],ascending=True)
reco_top10 = reco.head(10)
reco_top10 = reco_top10.join(recipes[['i','name']].set_index('i'))
reco_top10.columns = ['pred', 'name']
print('The flop 10 recommandations from Auto Encoder are :')
display(reco_top10[['name', 'pred']])

The flop 10 recommandations from Auto Encoder are :


,name,pred
i,,
99787,to die for crock pot roast,-0.305659
52334,yes virginia there is a great meatloaf,-0.301081
125637,pork chops yum yum,-0.289998
95864,how i got my family to eat spinach spinach ca...,-0.214532
22937,easy seven layer rice casserole,-0.206642
64311,poverty meal,-0.138964
96304,low fat cornbread,-0.104181
155731,sweet pickled banana peppers,-0.097441
31430,simmered round steak,-0.089497


# Get measure of performance for all train set

In [18]:
%timeit my_model.predict(full_matrix, verbose=1)

6384/6384 [==============================] - 1s 114us/sample
749 ms ± 10.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
